In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import random

In [3]:
"""Function for print to "Surfer *.grd ver. 6" text file format
 Nx - number of point in row, Ny - number of points in column,
 xMin, xMax - coordinates of left and right border in Ox dimension
 yMin, yMax - coordinates of down and up border in Oy dimension
 zMin, zMax - minimal and maximal values of grid (data)
 file - filename"""
def write_ascii_grid(Nx, Ny, xMin, xMax, yMin, yMax, zMin, zMax, data, file):
    with open(file, "w") as f:
        f.write("DSAA" + '\n')
        f.write("%s %s" % (Nx, Ny) + '\n')
        f.write("%s %s" % (xMin, xMax) + '\n')
        f.write("%s %s" % (yMin, yMax) + '\n')
        f.write("%s %s" % (zMin, zMax) + '\n')
        for row in data:
            f.write(' '.join([str(t) for t in row]) + '\n')

In [5]:
G = 6.673 # gravitational constant

""" Function Calculation of 3rd vertical derivative of gravity potential as model mascon effect"""
def pole(sig, rad, depth, ksi, eta):
    all3 = np.zeros((Nx, Ny))
    for i in range(0, Nx):
        for j in range(0, Ny):
            r = ((ksi - (i * dx + x0))**2 + (eta - (j * dy + y0))**2 + depth**2)**0.5
            all3[i,j] =  (G * sig * (4/3) * math.pi * (rad**3))*((15 * depth**3 / r**7) - (9 * depth / r**5)) 
    return all3

""" Function for creating a mask based on a threshold value"""
def my_mask(field, threshold): 
    mask = np.zeros((Nx, Ny))
    for j in range(0, Nx):
        for i in range(0, Ny):
            if (field[i,j] >= threshold):
                mask[i,j] = 1
    return mask

In [11]:
"""Parameters of data"""
Nx =  192 #160 #224 #192
dx = 100
Ny = Nx
dy = dx
x0 = -Nx/2*dx
y0 = x0

print(f"x0 = {x0}, dx = {dx}, Nx = {int(Nx)}")
print(f"y0 = {y0}, dy = {dy}, Ny = {int(Ny)}")

masc_n = 10 # Max number of mascons in one map (more than 2)
num_map = 67 # Number of maps with fixed number of mascons
n_models = (masc_n-1) * (num_map) # Number of models
print(f"Number of models: {n_models}")

rad_blur = 50 # Number of cells for smoothing the regional noise model
print(f"Averaging radius for smoothing: {rad_blur}")

rad_blur_hp = 10 # Number of cells for smoothing the high frequency noise model
print(f"Averaging radius for smoothing: {rad_blur_hp}")

x0 = -9600.0, dx = 100, Nx = 192
y0 = -9600.0, dy = 100, Ny = 192
Number of models: 603
Averaging radius for smoothing: 50
Averaging radius for smoothing: 10


In [13]:
"""Creating training Array"""

field = np.zeros((Nx, Ny))
field_res = np.zeros((Nx, Ny))
mask = np.zeros((Nx, Ny))
mask_res = np.zeros((Nx, Ny))
all_fields = np.zeros((0, Nx, Ny))
all_masks = np.zeros((0, Nx, Ny))

print('Start')

threshold = 0.2 # treshold for mask

for k in range(2, masc_n + 1): 
    for l in range(1, num_map + 1): 
        # Cycle for each mascon
        for m in range(1, k + 1): 
            qsig = 1 # You can leave it because we are already normalizing np.random.uniform(2.5, 3.51) 
            wrad = 500 # You can leave it because we are already normalizing np.random.randint(100, 700)
            qqdepth = np.random.randint(2*dx*3, (Nx*dx)/5)  # Random depth
            xi = random.uniform(x0, (x0 + Nx*dx)) 
            yi = random.uniform(y0, (y0 + Ny*dy)) 
            field = pole(qsig, wrad, qqdepth, xi, yi)
            field = np.clip(field, 0, field.max())
            field = (field - field.min())/(field.max() - field.min()) # Min-Max Scaling
            
            # Data clip
            field = np.clip(field, 0, np.random.uniform(0.6, 0.9))
            field[field < threshold] = 0
            
            field = (field - field.min())/(np.ptp(field)) # Min-Max Scaling
            field_res = field_res + field
            
            mask = my_mask(field, threshold) 
            mask_res = mask_res + mask
        
        # In case anomalies overlap
        for i in range(len(field)):
            for j in range(len(field[i])):
                if field_res[i][j] > 1:
                    field_res[i][j] = 1
                if mask[i][j] > 1:
                    mask[i][j] = 1
        #----------------------------------------------------------------
        
        Diap = np.ptp(field_res)

        Noise_mas = np.random.uniform((-Diap) / 100 * np.random.randint(25, 51), (Diap) / 100 * np.random.randint(25, 51), (Nx, Ny)) * 300
        Noise_mas_cv2 = cv2.blur(Noise_mas, (rad_blur, rad_blur))
        Noise_mas_cv2 = cv2.blur(Noise_mas_cv2, (rad_blur, rad_blur))

        Noise_mas_hp = np.random.uniform((Diap) / 100 * np.random.randint(25, 51), (-Diap) / 100 * np.random.randint(25, 51), (Nx, Ny)) * 3
        Noise_mas_hp_cv2 = cv2.blur(Noise_mas_hp, (rad_blur_hp, rad_blur_hp))
        Noise_mas_hp_cv2 = cv2.blur(Noise_mas_hp_cv2, (rad_blur_hp, rad_blur_hp))
                
        field_res_noise = field_res + Noise_mas_cv2 + Noise_mas_hp_cv2
        field_res_noise = (field_res_noise - field_res_noise.min())/(np.ptp(field_res_noise)) #Min-Max Scaling
        
        # Mask creation
        mask_res = np.clip(mask_res, 0, 1)
        
        all_fields = np.concatenate([all_fields, field_res_noise[np.newaxis, ...]], axis=0)
        field_res = np.zeros((Nx, Ny))
        all_masks = np.concatenate([all_masks, mask_res[np.newaxis, ...]], axis=0)
        mask_res = np.zeros((Nx, Ny))
    print((k - 1) * num_map,'/', n_models)

    # For image of model examples
    """plt.imshow(all_fields[(k - 1) * num_map - 5], aspect='auto', cmap='viridis')
    plt.colorbar(label='Field')
    plt.show()
    filename = 'Model'+str((k - 1) * num_map - 5)+'.grd'
    #write_ascii_grid(Nx, Ny, x0, -x0, y0, -y0, 0, 1, all_fields[(k - 1) * num_map - 5], filename)
    plt.imshow(all_masks[(k - 1) * num_map - 5], aspect='auto', cmap='viridis')
    plt.colorbar(label='Mask') 
    plt.show()
    filename = 'Mask'+str((k - 1) * num_map - 5)+'.grd'
    #write_ascii_grid(Nx, Ny, x0, -x0, y0, -y0, 0, 1, all_masks[(k - 1) * num_map - 5], filename)"""

np.save('../data/all_fields_n192_600.npy', all_fields)  # Save all fields
np.save('../data/all_masks_n192_600.npy', all_masks)    # Save all mask        


Start
67 / 603
134 / 603
201 / 603
268 / 603
335 / 603
402 / 603
469 / 603
536 / 603
603 / 603
